In [18]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, GlobalAveragePooling2D, Dense, Reshape, Multiply

# SENet

In [19]:
def squeeze_excite_block(input_tensor, ratio=16):
    channels = input_tensor.shape[-1]
    
    # Squeeze operation
    squeeze = GlobalAveragePooling2D()(input_tensor)
    
    # Excitation operation
    excitation = Dense(channels // ratio, activation='relu')(squeeze)
    excitation = Dense(channels, activation='sigmoid')(excitation)
    excitation = Reshape((1, 1, channels))(excitation)
    
    # Scale the input with the excitation weights
    scaled_input = Multiply()([input_tensor, excitation])
    
    return scaled_input

In [20]:
def create_senet_model(input_shape, num_classes):
    input_tensor = Input(shape=input_shape)
    
    # Example architecture, you may adjust it based on your requirements
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(input_tensor)
    x = squeeze_excite_block(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = squeeze_excite_block(x)
    
    x = GlobalAveragePooling2D()(x)
    x = Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=input_tensor, outputs=x)
    
    return model

In [21]:
# Example usage
input_shape = (224, 224, 12)  # Adjust based on your input image size and channels
num_classes = 10  # Adjust based on your classification task
senet_model = create_senet_model(input_shape, num_classes)

# Display model summary
senet_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 12 0                                            
__________________________________________________________________________________________________
conv2d_24 (Conv2D)              (None, 224, 224, 64) 6976        input_1[0][0]                    
__________________________________________________________________________________________________
global_average_pooling2d (Globa (None, 64)           0           conv2d_24[0][0]                  
__________________________________________________________________________________________________
dense (Dense)                   (None, 4)            260         global_average_pooling2d[0][0]   
______________________________________________________________________________________________

# ECA 

In [5]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, Reshape, Multiply, Conv1D, Dense

In [6]:
def eca_block(input_tensor, ratio=4):
    channels = input_tensor.shape[-1]

    # Squeeze operation
    squeeze = tf.reduce_mean(input_tensor, axis=[1, 2], keepdims=True)
    squeeze = Reshape((1, channels))(squeeze)

    # Excitation operation
    excitation = Conv1D(channels // ratio, kernel_size=3, padding='same', activation='relu')(squeeze)
    excitation = Conv1D(channels, kernel_size=3, padding='same', activation='sigmoid')(excitation)
    excitation = Reshape((1, 1, channels))(excitation)

    # Scale the input with the excitation weights
    scaled_input = Multiply()([input_tensor, excitation])

    return scaled_input

In [7]:
def create_ECANet_model(input_shape, num_classes):
    input_tensor = Input(shape=input_shape)
    
    # Example architecture with ECA blocks
    x = Conv2D(64, (3, 3), padding='same', activation='relu')(input_tensor)
    x = eca_block(x)
    x = Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = eca_block(x)
    
    x = Dense(num_classes, activation='softmax')(x)
    
    model = tf.keras.Model(inputs=input_tensor, outputs=x)
    
    return model

In [8]:
# Example usage
input_shape = (224, 224, 12)  # Adjust based on your input image size and channels
num_classes = 10  # Adjust based on your classification task
senet_model = create_ECANet_model(input_shape, num_classes)

# Display model summary
senet_model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 224, 224, 12 0                                            
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 224, 224, 64) 6976        input_2[0][0]                    
__________________________________________________________________________________________________
tf.math.reduce_mean (TFOpLambda (None, 1, 1, 64)     0           conv2d_2[0][0]                   
__________________________________________________________________________________________________
reshape_2 (Reshape)             (None, 1, 64)        0           tf.math.reduce_mean[0][0]        
____________________________________________________________________________________________